### Benchmark with HyperNetX

This notebook provides a comparison to HyperNetX for running several algorithms, calculating measures, and accessing properties. It is necessary to install the benchmarks [requirements file](../requirements/benchmarks.txt) to run this notebook.

In [1]:
import hypernetx as hnx
import xgi
import pandas as pd
import time

In [3]:
data = xgi.load_xgi_data("email-enron")
data.cleanup()
edgelist = xgi.to_hyperedge_list(data)
df = xgi.to_bipartite_pandas_dataframe(data)

In [4]:
start = time.time()
H1 = xgi.Hypergraph(edgelist)
H1 = H1.dual()
print(f"XGI edgelist construction time: {time.time() - start} sec")
print(f"The hypergraph has {H1.num_nodes} nodes and {H1.num_edges} edges.")

start = time.time()
H2 = hnx.Hypergraph(edgelist, static=True)
print(f"HyperNetX edgelist construction time: {time.time() - start} sec")
print(
    f"The hypergraph has {H2.number_of_nodes()} nodes and {H2.number_of_edges()} edges."
)

XGI edgelist construction time: 0.011233806610107422 sec
The hypergraph has 1459 nodes and 143 edges.
HyperNetX edgelist construction time: 0.023783206939697266 sec
The hypergraph has 143 nodes and 1459 edges.


In [5]:
start = time.time()
H1 = xgi.Hypergraph(df)
H1 = H1.dual()
print(f"XGI edgelist construction time: {time.time() - start} sec")
print(f"The hypergraph has {H1.num_nodes} nodes and {H1.num_edges} edges.")

start = time.time()
H2 = hnx.Hypergraph(df, static=True)
print(f"HyperNetX edgelist construction time: {time.time() - start} sec")
print(
    f"The hypergraph has {H2.number_of_nodes()} nodes and {H2.number_of_edges()} edges."
)

XGI edgelist construction time: 0.010957956314086914 sec
The hypergraph has 1459 nodes and 143 edges.
HyperNetX edgelist construction time: 0.2202610969543457 sec
The hypergraph has 1459 nodes and 143 edges.


In [27]:
print("XGI dual hypergraph construction time:")
%timeit H1.dual()

print("HyperNetX dual hypergraph construction time:")
%timeit H2.dual()

XGI dual hypergraph construction time:
6.28 ms ± 133 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
HyperNetX dual hypergraph construction time:
38.4 ms ± 393 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
print("XGI incidence matrix construction time:")
%timeit I1 = xgi.incidence_matrix(H1)
print("HyperNetX incidence matrix construction time:")
%timeit I2 = H2.incidence_matrix()

XGI incidence matrix construction time:
60.4 ms ± 1.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
HyperNetX incidence matrix construction time:
158 ns ± 0.0754 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
print("XGI adjacency matrix construction time:")
%timeit xgi.adjacency_matrix(H1)

print("HyperNetX adjacency matrix construction time:")
%timeit H2.adjacency_matrix()

XGI adjacency matrix construction time:
110 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
HyperNetX adjacency matrix construction time:
45 ms ± 88.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
print("XGI adjacency dictionary retrieval time:")
%timeit xgi.to_hyperedge_dict(H1)

print("XGI adjacency dictionary retrieval time:")
%timeit H2.incidence_dict

XGI adjacency dictionary retrieval time:
57.8 ms ± 330 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
XGI adjacency dictionary retrieval time:
198 ms ± 1.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
print("XGI node memberships retrieval time:")
start = time.time()
for node in H1.nodes:
    a = H1.nodes.memberships(node)
print(f"{time.time() - start} sec")

print("HyperNetX node memberships retrieval time:")
start = time.time()
for node in H2.nodes:
    a = H2.nodes.memberships[node]
print(f"{time.time() - start} sec")

XGI node memberships retrieval time:
0.003931760787963867 sec
HyperNetX node memberships retrieval time:
0.02753615379333496 sec


In [9]:
print("XGI hyperedge retrieval time:")
start = time.time()
for edge in H1.edges:
    a = H1.edges.members(edge)
print(f"{time.time() - start} sec")

print("HyperNetX hyperedge retrieval time:")
start = time.time()
for node in H2.nodes:
    a = H2.edges[edge]
print(f"{time.time() - start} sec")

XGI hyperedge retrieval time:
0.007923126220703125 sec
HyperNetX hyperedge retrieval time:
3.9082751274108887 sec


In [10]:
print("XGI time to determine connectedness:")
%timeit xgi.is_connected(H1)

print("HyperNetX time to determine connectedness:")
%timeit H2.is_connected()

XGI time to determine connectedness:
178 ms ± 4.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
HyperNetX time to determine connectedness:
272 ms ± 234 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
print("XGI time to determine connected components:")
%timeit [len(cc) for cc in xgi.connected_components(H1)]

print("HyperNetX time to determine connected components:")
%timeit [len(cc) for cc in H2.connected_components()]

XGI time to determine connected components:
181 ms ± 4.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
HyperNetX time to determine connected components:
284 ms ± 2.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
k1 = H1.nodes.degree.asdict()
k2 = H1.edges.size.asdict()

print("XGI time to construct Chung-Lu hypergraph:")
%timeit xgi.chung_lu_hypergraph(k1, k2)

print("HyperNetX time to construct Chung-Lu hypergraph:")
%timeit hnx.chung_lu_hypergraph(k1, k2)

XGI time to construct Chung-Lu hypergraph:
135 ms ± 2.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
HyperNetX time to construct Chung-Lu hypergraph:
5.98 s ± 21.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
